In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [2]:
def write_query(text):
    """
    Writes text to the query field.
    """
    query_field.click()
    try:
        #try_click(By.CLASS_NAME,'CodeMirror')
        query_field.click()
        #time.sleep(1)
        query_field.send_keys(Keys.CONTROL + "a")
        query_field.send_keys(text)
    except:
        pass

In [3]:
def initialize_patstat():
    print('Initializing Patstat, please wait a few minutes.')
    #browser exposes an executable file
    #Through Selenium test we will invoke the executable file which will then #invoke actual browser
    driver = webdriver.Firefox(executable_path="C:\Program Files\Mozilla Firefox\geckodriver.exe")
    # to maximize the browser window
    driver.maximize_window()
    #get method to launch the URL
    URL = driver.get("https://data.epo.org/expert-services/")
    # Close PatStat warning message
    time.sleep(2)
    button_Close = driver.find_element(By.ID,'dijit_form_Button_0_label')
    button_Close.click()
    time.sleep(2)
    
    # Connect to Patstat using the credentials
    
    username='contact@chaireeconomieduclimat.org'
    password='cLnI7ZDA'
    
    button_Username = driver.find_element(By.ID,'username')
    button_Username.click()
    button_Username.send_keys(username)
    button_Password = driver.find_element(By.ID,'password')
    button_Password.click()
    button_Password.send_keys(password)

    button_Log_in = driver.find_element(By.ID,'buttonLogin_label').click()
    time.sleep(5)

    # PatStat Spring 2022
    objects = driver.find_elements(By.CLASS_NAME, 'fullPart')
    objects = [el for el in objects if 'PATSTAT Online' in el.text]
    for obj in objects:
        try:
            obj.click()
            break
        except:
            pass

    # Go to the tab Table
    go_to_Table = driver.find_element(By.ID,'goToTableLink')
    go_to_Table.click()
    time.sleep(5)

    # Look for the query field in the elements of the page
    list_el = [el for el in driver.find_elements_by_xpath('//*') if 'tls201' in el.text]
    
    #If the function does not work, maybe the idenfication of the query field is wrong.
    #The second element (index 1) has been identified as the good one in previous trials.
    #Try to set query_el = range(len(list_el)) or identify the right element corresponding to the query field,
    #within 0 to len(list_el).
    
    query_field = list_el[1]
    
    # If TEST is displayed in the query field. Identification worked properly.
    write_query('TEST')
    print('Patstat initialized. Check if the word TEST is displayed in the query field.')
    
    return driver, query_field, list_el

In [4]:
def try_click(by_what, text_search, condition=None, display=True):
    """
    A function that tries to click elements of type 'by_what' which names contain 'text_search'
    """
    # Look for elements
    objects = driver.find_elements(by_what, text_search)
    
    if condition:
        objects = [el for el in objects if condition in el.text]
    
    count = 0
    clicked_object = None
    
    # Try to click on found elements
    for obj in objects:
        try:
            obj.click()
            clicked_object = obj
            break
        except:
            count += 1
            pass
    if count == len(objects) and display:
        print('No element clicked')
    return clicked_object

In [5]:
def download_all():
    
    time.sleep(5)
    button_download_menu = driver.find_element(By.ID,'download_text')
    button_download_menu.click()
    time.sleep(1)
    button_download_menu_2 = driver.find_element(By.ID,'downloadManagerMenuItem_text')
    button_download_menu_2.click()
    
    for download_button in driver.find_elements(By.XPATH,"//*[contains(@class, 'buttonDownloadSave') and not(contains(@class, 'buttonDownloadDelete'))]"):
        try:
            download_button.click()
        except:
            pass
        
        time.sleep(5)

In [6]:
def clear_all_downloads():
    
    button_download_menu = driver.find_element(By.ID,'download_text')
    button_download_menu.click()
    time.sleep(1)
    button_download_menu_2 = driver.find_element(By.ID,'downloadManagerMenuItem_text')
    button_download_menu_2.click()
    time.sleep(1)
    try_click(By.CLASS_NAME, 'dijitReset', condition='Clear list', display=False)
    
    # Clicking the confirmation dialogue
    try_click(By.CLASS_NAME, 'dijitReset', condition='Yes', display=False)
    
    # Closing the download manager
    close_dialog()

In [7]:
def close_dialog():
    for close_button in driver.find_elements(By.CLASS_NAME,'dijitDialogCloseIcon'):
        try:
            close_button.click()
        except:
            pass

In [8]:
def search():
    _ = try_click(By.ID,'queryBlockContentResultTable_btLaunchQuery')

In [9]:
def find_nb_rows():
    """
    Function to find the number of rows of a query. Uses the figure displayed on the top left.
    """
    try:
        idx = driver.page_source.find(' rows')
        text_around = driver.page_source[idx-100:idx]
        idx = text_around.find(""""onDblClick:stopEvent">""")
        text_around = text_around[idx+23:]
        nb_rows = int(text_around.replace("&nbsp;",''))
        return nb_rows
    
    except ValueError:
        print('No query found.')

In [10]:
def prepare_download():
    
    nb_rows = find_nb_rows()
    delta_row = 700000
    row = 1
    count_downloads = 0
    while row + delta_row < nb_rows:
        
        count_downloads += 1

        button_download_menu = driver.find_element(By.ID,'download_text')
        button_download_menu.click()
        time.sleep(1)
        button_download_menu_2 = driver.find_element(By.ID,'downloadMenuItem_text')
        button_download_menu_2.click()
        time.sleep(1)

        content_from = driver.find_element(By.ID,'dijit_form_TextBox_1')
        content_from.click()
        content_from.send_keys(Keys.CONTROL + "a") #select all
        content_from.send_keys(row)

        content_to = driver.find_element(By.ID,'dijit_form_TextBox_2')
        content_to.click()
        content_to.send_keys(Keys.CONTROL + "a") #select all
        content_to.send_keys(row+delta_row-1)

        button_OK = driver.find_element(By.ID,'btDownload_label').click()

        row += delta_row

        time.sleep(10)

    # Last query
    
    count_downloads += 1

    button_download_menu = driver.find_element(By.ID,'download_text')
    button_download_menu.click()
    time.sleep(1)
    button_download_menu_2 = driver.find_element(By.ID,'downloadMenuItem_text')
    button_download_menu_2.click()
    time.sleep(1)

    content_from = driver.find_element(By.ID,'dijit_form_TextBox_1')
    content_from.click()
    content_from.send_keys(Keys.CONTROL + "a") #select all
    content_from.send_keys(row)

    content_to = driver.find_element(By.ID,'dijit_form_TextBox_2')
    content_to.click()
    content_to.send_keys(Keys.CONTROL + "a") #select all
    content_to.send_keys(nb_rows)

    button_OK = driver.find_element(By.ID,'btDownload_label').click()
    
    print(count_downloads,'download(s) ready')
    return count_downloads

In [11]:
def wait(by_what, text_search):
    '''
    A function that waits.
    
    Typically, when interacting with Patstat, the screen might darken while some tasks are performed.
    This function tries to click a button continuously.
    If it is not available, the tasks are considered to be still in progress.
    '''
    
    #text_search = 'CodeMirror' and by_what = By.CLASS_NAME for a query to end
    #text_search = 'download_text' and by_what = By.ID for a download to end ??
    
    var = None
    while var is None:
        var = try_click(by_what, text_search,display=False)

In [12]:
def wait_for_download_preparation():
    '''
    A function that waits for the query to end.
    
    Basically, when a query is queued, the screen darkens and a message 'Search in progress' pops.
    This prevents clicking on the search button again.
    This function tries to click the button continuously.
    If the search button is not available, the query is considered still in progress.
    '''
    var = None
    while var is None:
        var = try_click(By.CLASS_NAME,'CodeMirror',display=False)

In the following example, the script launches three different queries. One for the patent count between 1990 and 1999, one between 2000 and 2009, and the last one between 2010 and 2019. Notice that this could have been done in one shot, from 1990 to 2019, but this just serves as an example here.

In [13]:
driver, query_field, fields = initialize_patstat()

Initializing Patstat, please wait a few minutes.


StaleElementReferenceException: Message: The element reference of [object String] "{\"element-6066-11e4-a52e-4f735466cecf\":\"ab8794fa-d3bc-434c-9d38-8fa96849a5c3\"}" is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed


In [90]:
def example_query(date_start, date_end):
    text = f"""
SELECT a.appln_filing_year, COUNT(DISTINCT a.appln_id) AS NumberOfPatentApplications
FROM tls201_appln a
JOIN tls209_appln_ipc i ON a.appln_id = i.appln_id
WHERE i.ipc_class_symbol LIKE 'B08B%' -- Change IPC symbol here
AND a.appln_filing_year BETWEEN {str(date_start)} AND {str(date_end)} -- Define year range here
GROUP BY a.appln_filing_year
ORDER BY a.appln_filing_year
"""
    return text

In [91]:
print(example_query(2012,2013))


SELECT a.appln_filing_year, COUNT(DISTINCT a.appln_id) AS NumberOfPatentApplications
FROM tls201_appln a
JOIN tls209_appln_ipc i ON a.appln_id = i.appln_id
WHERE i.ipc_class_symbol LIKE 'B08B%' -- Change IPC symbol here
AND a.appln_filing_year BETWEEN 2012 AND 2013 -- Define year range here
GROUP BY a.appln_filing_year
ORDER BY a.appln_filing_year



In [89]:
for date_start in [1990,2000,2010]:

    query_field.click()
    write_query(example_query(date_start, date_start+9))
    search()
    # Wait for query
    wait(By.CLASS_NAME, 'CodeMirror')
    print(find_nb_rows(), 'entries found')
    count_downloads = prepare_download()
    # Wait for preparation
    #wait(By.ID, 'download_text')
    download_all()
    close_dialog()
    
    # rename the downloads
    download_path = Path('C:/Users/Dhorne/Downloads')
    
    time.sleep(10)
    downloaded_files = [file for file in download_path.iterdir() if 'resulttable' in file.name]
    
    identifier = str(date_start)+'_'+str(date_start+9)

    if count_downloads != len(downloaded_files):
        print('Found', len(downloaded_files),'files, expected', count_downloads)
    else:
        if len(downloaded_files) == 1:
            os.rename(downloaded_files[0], str(downloaded_files[0].parent)+'/'+identifier+downloaded_files[0].suffix)
        else:
            count = 1
            for file in downloaded_files:
                os.rename(file, str(file.parent)+'/'+identifier+'_'+str(count)+file.suffix)
                count += 1
                
    clear_all_downloads()

No element clicked
10 entries found


ElementNotInteractableException: Message: Element <td id="downloadMenuItem_text" class="dijitReset dijitMenuItemLabel"> could not be scrolled into view
